In [1]:
import sys
import os
from dotenv import load_dotenv
from datetime import datetime
import pandas as pd



load_dotenv()


# Set the root directory of your project
project_root = '/Users/rodolfocacacho/Documents/Documents/MAI/Master Thesis/Code/rag_project'

os.chdir(project_root)

# Add the root directory to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

In [2]:
from utils.MySQLDB_manager import MySQLDB
from config import (DB_NAME,CONFIG_SQL_DB,
                    SQL_VOCAB_BM25_TABLE,SQL_PROMPTS_TABLE)
from rag_server.modules.rag_retrieval_bm25 import bm25_query_vectors,bm25_query_vectors_original,preprocess_vocabulary,retrieve_context,get_dates_documents
from utils.embedding_handler import EmbeddingHandler
from config import EMBEDDING_MODEL,EMBEDDING_MODEL_RET_TASK,EMBEDDING_MODEL_API,INDEX_NAME,EMBEDDING_MODEL_DIM,SQL_CHUNK_TABLE,METADATA_FILE_PATH
from utils.pinecone_hybrid_connector import PineconeDBConnectorHybrid
API_PINE_CONE = os.getenv('API_PINE_CONE')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rodolfocacacho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
emb = EmbeddingHandler(model_name=EMBEDDING_MODEL,
                       use_api=EMBEDDING_MODEL_API,
                       task=EMBEDDING_MODEL_RET_TASK)

sql_con = MySQLDB(config=CONFIG_SQL_DB,
                  database_name=DB_NAME)

vec_con = PineconeDBConnectorHybrid(index_name=INDEX_NAME,
                                    api_key=API_PINE_CONE,
                                    dimension=EMBEDDING_MODEL_DIM)

vocab = sql_con.get_all_records_as_dict(SQL_VOCAB_BM25_TABLE)
vocabbm25 = preprocess_vocabulary(vocab)
vocab_ids = {}
for entry in vocab:
    word = entry['word'].lower()
    word_id = entry['id']
    
    # Add the original word
    vocab_ids[word_id] = {'original_word': word}


Embedding Dimension: 1024
Using existing Pinecone index: german-semantic-v3-clean-250


In [4]:
query_o = """Wie hoch ist die Förderung für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?"""
queries_alt = ["Welche Fördermittel gibt es für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?", "Wie hoch ist die finanzielle Unterstützung für die Modernisierung von Beleuchtungssystemen in Nichtwohngebäuden?", "Welche Zuschüsse existieren für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?"]

In [5]:
retrieved_date = "23/11/2024"
retrieved_date = datetime.strptime(retrieved_date, '%d/%m/%Y')
date_database = pd.read_csv(METADATA_FILE_PATH)
dates_project = get_dates_documents(retrieved_date,date_database)


results = retrieve_context(vector_db_connector=vec_con,
                           sql_connector=sql_con,embed_handler=emb,embed_task=EMBEDDING_MODEL_RET_TASK,
                           sql_table=SQL_CHUNK_TABLE,vocab_table=SQL_VOCAB_BM25_TABLE,alpha_value=0.7,query=query_o,
                           gen_prompts=queries_alt,keyterms="HALLO",retrieve_date_documents=dates_project,max_results_query=20,max_results=5)

Reading Vocabulary
Done reading vocabulary
['Welche Fördermittel gibt es für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?', 'Wie hoch ist die finanzielle Unterstützung für die Modernisierung von Beleuchtungssystemen in Nichtwohngebäuden?', 'Welche Zuschüsse existieren für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?'] 3
Wie hoch ist die Förderung für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?
['Wie hoch ist die Förderung für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?', 'Welche Fördermittel gibt es für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?', 'Wie hoch ist die finanzielle Unterstützung für die Modernisierung von Beleuchtungssystemen in Nichtwohngebäuden?', 'Welche Zuschüsse existieren für die Erneuerung von Beleuchtungssystemen in Nichtwohngebäuden?'] 4
Sparse vectors
['hoch förderung', 'förderung erneuerung', 'erneuerung beleuchtungssysteme', 'beleuchtungssysteme nichtwohngebäuden', 'hoch för

In [4]:
query1 = """Welche Vorlauftemperatur ist maximal zulässig, damit aktuelle Heizflächen gefördert werden?"""
query2 = """Welche U-Werte müssen Bauteile haben, um gefördert zu werden"""
query3 = """Welche technischen Anforderungen bestehen an die Beleuchtung"""
query4 = """Welche technischen Anforderungen bestehen an Heizflächen?"""
query5 = """'Welche Regelung besteht bezüglich der Förderhöchstgrenze für Maßnahmen nach Nummer 5.3 (Wärmeerzeugung) für Nichtwohngebäude?'"""
queries = [query1,query2,query3,query4,query5]
# query = """Bei Wohngebäuden: Welcher U-Wert gilt bei Kellerdecken für eine Förderung"""


In [5]:
prompts = sql_con.get_all_records_as_dict(SQL_PROMPTS_TABLE)
user_queries = [i['question'] for i in prompts]

In [6]:
sparse_vector_queries = bm25_query_vectors(queries,vocabbm25)
sparse_vector_queries_original = bm25_query_vectors_original(queries,vocabbm25)



In [7]:
for i,vector in enumerate(sparse_vector_queries):
    print(f"Query {i}:{queries[i]}\n{vector}\n{sparse_vector_queries_original[i]}")
    words = [vocab_ids[word]['original_word'] for word in vector['indices']]
    print(f'{words}\n')

Query 0:Welche Vorlauftemperatur ist maximal zulässig, damit aktuelle Heizflächen gefördert werden?
{'indices': [257, 1294, 275, 744, 771, 188], 'values': [4.7836, 3.0829, 3.61352, 3.43697, 7.49165, 1.79847]}
{'indices': [257, 1294, 275, 744, 771, 188], 'values': [4.7836, 3.0829, 3.61352, 3.43697, 7.49165, 1.79847]}
['vorlauftemperatur', 'maximal', 'zulässig', 'aktuell', 'heizfläch', 'fördern']

Query 1:Welche U-Werte müssen Bauteile haben, um gefördert zu werden
{'indices': [1874, 208, 1392, 188], 'values': [4.09045, 2.22465, 3.52135, 1.79847]}
{'indices': [1874, 208, 1392, 188], 'values': [4.09045, 2.22465, 3.52135, 1.79847]}
['uwert', 'müssen', 'bauteil', 'fördern']

Query 2:Welche technischen Anforderungen bestehen an die Beleuchtung
{'indices': [201, 146, 41, 2117], 'values': [1.19147, 2.43859, 2.96444, 4.97934]}
{'indices': [201, 146, 41, 2117], 'values': [1.19147, 2.43859, 2.96444, 4.97934]}
['technisch', 'anforderung', 'bestehen', 'beleuchtung']

Query 3:Welche technischen Anfo

In [23]:
for i,vector in enumerate(sparse_vector_queries_t):
    if len(vector['indices']) > len(sparse_vector_queries_original_t[i]['indices']):
        print(f"Query {i}: {user_queries[i]} - {len(vector['indices']) - len(sparse_vector_queries_original_t[i]['indices'])}\n{vector}\n{sparse_vector_queries_original_t[i]}")
        words = [vocab_ids[word]['original_word'] for word in vector['indices']]
        print(f'{words}\n')


Query 9: Welche U-Wert-Anforderungen haben denkmalgeschützte Fenster? - 1
{'indices': [6050, 690, 483], 'values': [8.59026, 5.59453, 3.29947]}
{'indices': [6050, 483], 'values': [8.59026, 3.29947]}
['u-wert-anforderung', 'denkmal', 'fenster']

Query 10: Bezieht sich die Antwort auf denkmalgeschützte Fenster oder auf Fenster in denkmalgeschützten Gebäuden? - 1
{'indices': [551, 4366, 690, 483, 6], 'values': [3.50885, 4.85259, 7.992185714285714, 4.713528571428571, 0.889963]}
{'indices': [551, 4366, 483, 6], 'values': [3.50885, 4.85259, 4.713528571428571, 0.889963]}
['beziehen', 'antwort', 'denkmal', 'fenster', 'gebäude']

Query 11: Welche U-Wert-Anforderungen haben denkmalgeschützte Fenster? - 1
{'indices': [6050, 690, 483], 'values': [8.59026, 5.59453, 3.29947]}
{'indices': [6050, 483], 'values': [8.59026, 3.29947]}
['u-wert-anforderung', 'denkmal', 'fenster']

Query 12: Welche U-Wert-Anforderungen haben denkmalgeschützte Fenster? - 1
{'indices': [6050, 690, 483], 'values': [8.59026, 5.

In [9]:
results[2]

['35.6',
 '76.50',
 '35.68',
 '9.6',
 '35.70',
 '76.68',
 '2.82',
 '2.24',
 '76.61',
 '76.32']

In [11]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(EMBEDDING_MODEL, trust_remote_code=True)



In [13]:
embedding_model.encode(queries,task = EMBEDDING_MODEL_RET_TASK,prompt_name=EMBEDDING_MODEL_RET_TASK,batch_size=16,show_progress_bar=True)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

array([[-0.04297601, -0.00960067,  0.0041041 , ...,  0.00590087,
         0.00372966,  0.00604213],
       [-0.04838692,  0.06883916,  0.09488188, ...,  0.03442793,
        -0.00346939,  0.03679278],
       [-0.0015143 ,  0.12412091,  0.07085968, ...,  0.03048928,
        -0.00215709,  0.03923195]], dtype=float32)

In [1]:
test = ["35.93", "76.45", "2.19", "2.20", "2.8", "35.41", "35.97", "35.101", "76.6", "35.74", "2.22", "2.9", "35.58", "2.92", "35.126", "76.7", "2.24", "35.23", "76.68", "76.46", "2.98", "76.49", "35.128", "2.29", "2.32", "35.53", "76.5", "35.107", "35.7", "35.2", "35.68", "35.44", "35.102", "35.62", "2.23", "35.25", "35.119", "2.21", "76.64", "56.11"]

len(test)

40